In [1]:
import random
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [2]:
def sample_gen(n ) :
    arr = []
    for j in range(n) :
        arr.append([np.random.normal(0,1.0) , np.random.normal(0,1.0)])
    return np.array(arr).T
def get_corr_sample(sample , rho) :
    alpha = (np.sqrt(1+rho) + np.sqrt(1-rho))/2
    beta = (np.sqrt(1+rho) - np.sqrt(1-rho))/2
    A_rho = [[alpha , beta],[beta , alpha]]
    return A_rho@sample
def T(X) :
    return np.sum(X[0]*X[1])/len(X[0])
def T_2(X) :
    return np.sum((X[0]-X[1])**2)/len(X[0])/2
def get_W(n , criterion , num_samples ,rho_arr) :
    W = []
    for rho in rho_arr :
        count_reject = []
        for _ in range(num_samples) :
            sample = sample_gen(n)
            sample = get_corr_sample(sample , rho)
            count_reject.append(criterion(sample))
        W.append( sum(count_reject) / num_samples)
    return W
def crit_1(sample)  :
    return np.abs(T(sample)) > 20/n
def crit_2(sample) :
    return np.abs(T_2(sample)-1) > 1.96*np.sqrt(2)/np.sqrt(n)

In [3]:
def get_p_values(num_samples ,rho , n=100 ) :
    p_values = []
    sigma = np.sqrt((1+rho**2)/n)
    for _ in range(num_samples) :
        sample = sample_gen(n)
        sample = get_corr_sample(sample , rho)
        p_values.append(norm.cdf( T(sample) , loc=rho, scale=sigma ))
    return p_values    

In [5]:
n = 100
rho_1  = 0
rho_2 = 0.2
num_samples = 500

p_values_1 = get_p_values(num_samples,rho_1,n)
p_values_2 = get_p_values(num_samples,rho_2,n)


In [ ]:
def FDR(p_values_1 , )